In [4]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

Collaborator: Kaleem

# Problem 1

### `1a` - Dynamics of a Future using Constant Elasticity of Variance (CEV) model
Let $S$ be a futures price, assumed to have constant elasticity of variance (CEV) dynamics. The CEV model is a stochastic volatility model that attempts to capture stochastic volatility and the leverage effect. $S$ follows a geometric Brownian motion with a volatility that depends on $S_t$ itself, mathemtically, it evolves according to the following stochastic differential equation:
$$\mathrm{d}S_t=\sigma S_t^{1+\alpha}\mathrm{d}W_t,\quad S_0=100$$
- $\sigma, \alpha$ are constants, in this problem
- $r$ is the interest rate on the bank account, in this problem
- $R_t=0$
- $A_t=\sigma S_t^{1+\alpha}$

Let $C(S_t, t)$ be the time-$t$ no-arbitrage price of an European put on $S$, with strike $K$ and expiry $T$. 

The PDE for $C(S, t)$ is derived as follows. Definitions - 
- $\partial _t C(S_t, t):=\partial C(S_t, t)/\partial t$
- $\partial_{s} C(S_t, t):=\partial C(S_t, t)/\partial S_t$
- $\partial_{s} C(S_t, t):=\partial^2 C(S_t, t)/\partial S_t^2$

By Ito’s formula - 
$$\begin{aligned}
dC(S_t, t) &= \left[ \partial_t C(S_t, t) + R_t \partial_{s} C(S_t, t) + \frac{A_t^2}{2} \partial_{ss} C(S_t, t) \right]dt + A_t \partial_{s} C(S_t, t) dW_t \\
\end{aligned}$$

Even though $S$ is a futures price (which has drift coefficient 0), prices of European options on $S$ still have drift coefficient $r$, so, we arrive at the following PDE for $C(r_t, t)$ -
$$\frac{\partial C(S_t, t)}{\partial t} + \frac{\sigma^2 S_t^{2+2\alpha}}{2} \frac{\partial^2 C(S_t, t)}{\partial s^2} - rC(S_t, t) =0$$

The terminal condition for $C(S, t)$ is - 
$$C(S_t, t) = (K-S)^+$$


In [5]:
class CEV:

    def __init__(self,volcoeff,alpha,rGrow,r,S0):
        self.volcoeff = volcoeff
        self.alpha = alpha
        self.rGrow = rGrow
        self.r = r
        self.S0 = S0


In [6]:
hw4dynamics = CEV(volcoeff=3, alpha=-0.5, rGrow=0, r=0.05, S0=100)

In [7]:
class Put:

    def __init__(self,T,K):
        self.T = T;
        self.K = K;

In [8]:
hw4contract=Put(T=0.25, K=100)

### Pricer (Finite Difference Scheme) for the Interest Rate Derivative

There are several different versions of a finite difference (FD) scheme -
- In the **standard/ explicit FD** scheme, $C_n^j$ is determined by $C_{n+1}^{j+1}$, $C_{n+1}^{j}$, and $C_{n+1}^{j-1}$, allowing one to solve for $C_n^j$ directly via backtracking
- In the **implicit FD** scheme, $C_n^j$ is determined by $C_{n}^{j+1}$, $C_{n+1}^{j}$, and $C_{n}^{j-1}$, requiring the setup of a system of equations via backtracking
- In the **Crank-Nicolson** scheme, an off-grid midpoint is determined by the combination of the three points to the left of it and three points to the right of it

Comparison between them 
- The **standard/ explicit FD** scheme, has accuracy $O(\Delta t) +O(\Delta x)^2$, is not unconditionally stable, and does not require solving linear systems
- The **implicit FD** scheme, has accuracy $O(\Delta t) +O(\Delta x)^2$, is unconditionally stable, and requires solving linear systems
- The **Crank-Nicolson** scheme, has accuracy $O(\Delta t)^2 +O(\Delta x)^2$, is unconditionally stable, and requires solving linear systems, making it the most accurate and computationally complex scheme out of all three

### `1b` Crank-Nicolson scheme
In the Crank-Nicolson scheme, we want to solve for $C(x,t)$ the full PDE
$$\frac{\partial C}{\partial t}+f(x,t)\frac{\partial^2C}{\partial x^2}+g(x,t)\frac{\partial C}{\partial x}+h(x,t)C=0$$
* $f(x,t)=\frac{\sigma^2 S_t^{2+2\alpha}}{2}$
* $g(x,t)=0$
* $h(x,t)=-r$
* given terminal condition $C(S_t, t) = (K-S)^+$
* given specific values $r=0.05$, $\sigma=3$, $\alpha=-0.5$. And $K=100$, $T=0.25$.

Now,
- $f_n^j := f(x_j, t_n)=\frac{\sigma^2 S_t^{2+2\alpha}}{2}$
- $g_n^j := g(x_j, t_n)=0$
- $h_n^j := h(x_j, t_n)=-r$
- $f_{n+1}^j := f(x_j, t_{n+1})=\frac{\sigma^2 S_t^{2+2\alpha}}{2}$
- $g_{n+1}^j := g(x_j, t_{n+1})=0$
- $h_{n+1}^j := h(x_j, t_{n+1})=-r$

Discretize the PDE, note that now the differential terms in space are approximated by averaging the terms from both $n$ and $n+1$ time steps, instead of just the $n+1$ side (explicit scheme) or just the $n$ side (implicit scheme), whereas the differential terms in time stay the same -
$$\begin{aligned}
\frac{C_{n+1}^{j}-C_{n}^{j}}{\Delta t}+\frac{1}{2}\left[f_{n}^{j}\frac{C_{n}^{j+1}-2C_{n}^{j}+C_{n}^{j-1}}{(\Delta x)^{2}}+g_{n}^{j}\frac{C_{n}^{j+1}-C_{n}^{j-1}}{2\Delta x}+h_{n}^{j}C_{n}^{j}\right] \\
+\frac12\left[f_{n+1}^j\frac{C_{n+1}^{j+1}-2C_{n+1}^j+C_{n+1}^{j-1}}{(\Delta x)^2}+g_{n+1}^j\frac{C_{n+1}^{j+1}-C_{n+1}^{j-1}}{2\Delta x}+h_{n+1}^jC_{n+1}^j\right]=0
\end{aligned}$$

With terminal conditions given, we know $C_{n+1}$ and we solve for $C_n$. Then for $j=-J+1, ... J-1$,
$$-F_n^jC_n^{j+1}+(1+G_n^j)C_n^j-H_n^jC_n^{j-1}=F_{n+1}^jC_{n+1}^{j+1}+(1-G_{n+1}^j)C_{n+1}^j+H_{n+1}^jC_{n+1}^{j-1}$$
where 
$$\begin{aligned}
&F_{n}^{j} \begin{aligned}&:=\frac12\frac{\Delta t}{(\Delta x)^2}f_n^j+\frac14\frac{\Delta t}{\Delta x}g_n^j\end{aligned} \\
&G_n^j :=\frac{\Delta t}{(\Delta x)^2}f_n^j-\frac{\Delta t}2h_n^j \\
&H_n^j \begin{aligned}&:=\frac12\frac{\Delta t}{(\Delta x)^2}f_n^j-\frac14\frac{\Delta t}{\Delta x}g_n^j.\end{aligned} 
\end{aligned}$$

In matrix form, 
$$\left.\left(\begin{array}{ccccc}1+G_n^{J-1}&-H_n^{J-1}&&&\\-F_n^{J-2}&1+G_n^{J-2}&-H_n^{J-2}&&\\&-F_n^{J-3}&1+G_n^{J-3}&-H_n^{J-3}&\\&&\cdots&\cdots&\cdots\\&&&-F_n^{-J+1}&1+G_n^{-J+1}\end{array}\right.\right)\left(\begin{array}{c}C_n^{J-1}\\C_n^{J-2}\\\vdots\\\vdots\\\vdots\\C_n^{-J+1}\end{array}\right)+\left(\begin{array}{c}-F_n^{J-1}C_n^J\\0\\\vdots\\\vdots\\0\\-H_n^{-J+1}C_n^{-J}\end{array}\right)$$
$$\left.=\left(\begin{array}{ccccc}1-G_{n+1}^{J-1}&H_{n+1}^{J-1}&&&\\F_{n+1}^{J-2}&1-G_{n+1}^{J-2}&H_{n+1}^{J-2}&&\\&F_{n+1}^{J-3}&1-G_{n+1}^{J-3}&H_{n+1}^{J-3}&\\&&\cdots&\cdots&\cdots\\&&&F_{n+1}^{-J+1}&1-G_{n+1}^{-J+1}\end{array}\right.\right)\left(\begin{array}{c}C_{n+1}^{J-1}\\C_{n+1}^{J-2}\\C_{n+1}^{J-2}\\\vdots\\\vdots\\\vdots\\C_{n+1}^{-J+1}\end{array}\right)+\left(\begin{array}{c}F_{n+1}^{J-1}C_{n+1}^J\\0\\\vdots\\\vdots\\0\\H_{n+1}^{-J+1}C_{n+1}^{-J}\end{array}\right)$$

The spsolve function from scipy.sparse.linalg is used to solve linear systems of the form $Ax=b$, where $A$ is a sparse matrix and $b$ is a dense vector. It efficiently computes the solution $x$ using appropriate sparse matrix solvers.

In [9]:
class FD_CrankNicolson_Engine:

    def __init__(self,SMax,SMin,deltaS,deltat):
        self.SMax=SMax
        self.SMin=SMin
        self.deltaS=deltaS
        self.deltat=deltat


    #You complete the coding of this function:

    def price_put_CEV(self,contract,dynamics):
    # returns array of all initial spots,
    # and the corresponding array of put prices

        alpha, r, rGrow, volcoeff = dynamics.alpha, dynamics.r, dynamics.rGrow, dynamics.volcoeff

        # SMin and SMax denote the smallest and largest S in the _interior_.
        # The boundary conditions are imposed one level _beyond_,
        # e.g. at S_lowboundary=SMin-deltaS, not at SMin.
        # To relate to lecture notation, S_lowboundary is S_{-J}
        # whereas SMin is S_{-J+1}

        N=round(contract.T/self.deltat)
        if abs(N-contract.T/self.deltat)>1e-12:
            raise ValueError('Bad time step')
        numS=round((self.SMax-self.SMin)/self.deltaS)+1
        if abs(numS-(self.SMax-self.SMin)/self.deltaS-1)>1e-12:
            raise ValueError('Bad time step')
        S=np.linspace(self.SMax,self.SMin,numS)    #The FIRST indices in this array are for HIGH levels of S
        S_lowboundary=self.SMin-self.deltaS

        putprice=np.maximum(contract.K-S,0)

        ratio1 = self.deltat/self.deltaS
        ratio2 = self.deltat/self.deltaS**2
        f = np.full_like(S, 0.5 * volcoeff**2 * S**(2 + 2*alpha))
        g = np.full_like(S, 0)
        h = np.full_like(S, -r)
        F = 0.5 * ratio2 * f + 0.25 * ratio1 * g
        G =       ratio2 * f - 0.50 * self.deltat * h
        H = 0.5 * ratio2 * f - 0.25 * ratio1 * g

        #Right-hand-side matrix
        RHSmatrix = diags([H[:-1], 1-G, F[1:]], [1, 0, -1], shape=(numS,numS), format="csr")

        #Left-hand-side matrix
        LHSmatrix = diags([-H[:-1], 1+G, -F[1:]], [1, 0, -1], shape=(numS,numS), format="csr")
        # diags creates SPARSE matrices

        for t in np.arange(N-1,-1,-1)*self.deltat:

            rhs = RHSmatrix * putprice

            #Now let's add the boundary condition vectors.
            #They are nonzero only in the last component, as top evaluates to zero:
            rhs[-1] +=  2 * H[-1] * (contract.K - S_lowboundary)

            putprice = spsolve(LHSmatrix, rhs)
            # numpy.linalg.solve, which expects arrays as inputs,
            # is fine for small matrix equations, and for matrix equations without special structure.
            # But for large matrix equations in which the matrix has special structure,
            # we may want a more intelligent solver that can run faster
            # by taking advantage of the special structure of the matrix.
            # Specifically, in this case, let's try to use a solver that recognizes the SPARSE MATRIX structure.
            # Try spsolve, imported from scipy.sparse.linalg

            putprice = np.maximum(putprice, contract.K-S)

        return(S, putprice)

In [10]:
hw4FD = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)

In [11]:
(S0_all, putprice) = hw4FD.price_put_CEV(hw4contract,hw4dynamics)

In [12]:
# pricer_put_CEV_CrankNicolson gives us option prices for ALL S0 from SMin to SMax
# But let's display only for a few S0 near 100:

displayStart = hw4dynamics.S0 - hw4FD.deltaS * 1.5
displayEnd   = hw4dynamics.S0 + hw4FD.deltaS * 1.5
displayrows  = (S0_all>displayStart) & (S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all, putprice), axis=1)[displayrows])

[[100.1      5.8704]
 [100.       5.9183]
 [ 99.9      5.9665]]


### `1c` Time-0 Greeks
Compute numerically the time-0 delta and gamma of the put in (b).

At $(S, t)$,
$$\Delta:=\frac{\partial C}{\partial S}\approx\frac{C(S+\Delta S,t)-C(S-\Delta S,t)}{2\Delta S}$$
$$\Gamma:=\frac{\partial^2C}{\partial S^2}\approx\frac{C(S+\Delta S,t)-2C(S,t)+C(S-\Delta S,t)}{(\Delta S)^2}$$

In [30]:
delta_0 = (np.roll(putprice, 1) - np.roll(putprice, -1)) / (2 * hw4FD.deltaS)
gamma_0 = (np.roll(putprice, 1) - 2 * putprice + np.roll(putprice, -1)) / (hw4FD.deltaS**2)
np.set_printoptions(precision=4, suppress=True)
print("Time-0 Delta:")
print(np.stack((S0_all, delta_0), axis=1)[displayrows])
print("Time-0 Gamma:")
print(np.stack((S0_all, gamma_0), axis=1)[displayrows])

Time-0 Delta:
[[100.1     -0.478 ]
 [100.      -0.4806]
 [ 99.9     -0.4833]]
Time-0 Gamma:
[[100.1      0.0264]
 [100.       0.0264]
 [ 99.9      0.0264]]


### `1d` Assuming Black-Scholes
Using exactly the same FD CrankNicolson.price put CEV function as in (b) – meaning that you can change the input passed into the function, but cannot change the function’s code – find the time-0 price of the American put in (b), but assuming Black-Scholes dynamics for S with volatility 0.30 and interest rate 0.05 and S0 = 100.

In [31]:
# Black-Scholes - geometric brownian motion dynamics, power of S_t is 1
hw4dynamics_BS = CEV(volcoeff=0.3, alpha=0, rGrow=0, r=0.05, S0=100)

In [32]:
hw4FD_BS = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)

In [33]:
(S0_all_BS, putprice_BS) = hw4FD_BS.price_put_CEV(hw4contract, hw4dynamics_BS)

In [35]:
print("Put pricing under Black-Scholes:")
print(np.stack((S0_all_BS, putprice_BS), axis=1)[displayrows])

Put pricing under Black-Scholes:
[[100.1      5.8705]
 [100.       5.9169]
 [ 99.9      5.9636]]


# Problem 2

### `2a`
Out of a deck of 52 cards, there are 44 unrevealed cards, of which 10 would make Jamie the better hand, and the other 34 would make Patrik the better hand. Reveal one more card. Say the money in the pot is $1. Let $X$ be the payoff of the pot that Patrik will collect. Let $p(X)$ be the probability that the card is in Patrik's favor. The expectation and standard deviation of the fraction of $X$, when the last card (the “river”) is dealt in the usual way, is - 
$$E[X]=\sum X_i \cdot p(X_i)= 1\times \frac{34}{44}+0\times \frac{10}{44}= \frac{34}{44}$$
$$\begin{aligned}
Var(X)&=E[X^2]- E[X]^2 \\
&=\sum \left[X_i^2 \cdot p(X_i)\right] - \left[\sum X_i \cdot p(X_i)\right]^2 \\
&= \left[1^2\times \frac{34}{44}+0^2\times \frac{10}{44}\right] - \left[1\times \frac{34}{44}+0\times \frac{10}{44}\right]^2 \\
&= \frac{34}{44}-\left(\frac{34}{44}\right)^2
\end{aligned}$$

$$\sigma(X) = \sqrt{Var(X)}=0.41907

### `2b`
Suppose instead that the river is dealt three times. Denote a card favoring Jamie as $J$ and a card favoring Patrik as $P$, then there are three distinct situations involving $JP$, namely $JJJ, JJP, JPP, PPP$ with payoff $0, 1/3, 2/3, 1$ for Patrik. 
Let $p$ be the probability that Patrik wins in one iteration is, 
$$p =\frac{34}{44}$$ 
Let $q$ be the probability that Patrik loses in one iteration is, 
$$q =\frac{10}{44}$$ 
The probability that Patrik wins exactly $x$ iterations out of the three is, 
$$p(X) =\binom{3}{x} p^x (1-p)^{3-x} = \binom{3}{x} \left(\frac{34}{44}\right)^x \left(\frac{10}{44}\right)^{3-x} $$ 
 
Due to $JP$ agreeing to “running the river three times”, with replacement, the new expetation of Patrik's winning is,
$$\begin{aligned}E[X]&=\sum X_i \cdot p(X_i) \\
&= 0 \cdot \binom{3}{0} \left(\frac{34}{44}\right)^0 \left(\frac{10}{44}\right)^{3} + \frac{1}{3} \cdot \binom{3}{1} \left(\frac{34}{44}\right)^1 \left(\frac{10}{44}\right)^{2} \\
&+ \frac{2}{3} \cdot \binom{3}{2} \left(\frac{34}{44}\right)^2 \left(\frac{10}{44}\right)^{1} + \frac{3}{3} \cdot \binom{3}{3} \left(\frac{34}{44}\right)^3 \left(\frac{10}{44}\right)^{0} 
 \\
&= 0.773 \\
\end{aligned}$$

The new standard deviation of Patrik's winning is,
$$\begin{aligned}
Var(X)&=E[X^2]- E[X]^2 \\
&=\sum \left[X_i^2 \cdot p(X_i)\right] - \left[\sum X_i \cdot p(X_i)\right]^2 \\
\end{aligned}$$

$$\sigma(X) = \sqrt{Var(X)}=0.242

In [44]:
p = 34/44
q = 10/44
n = 3

mu = 0
for i in range(n+1):
    pi = np.math.comb(n, i) * p**i * q**(n-i)
    mu += (i/3) * pi
    
print("Mean is: ", mu)


Mean is:  0.7727272727272727


/var/folders/h6/9mpjzmxd77l7493g4210phz00000gn/T/ipykernel_1008/3720673284.py:7: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  pi = np.math.comb(n, i) * p**i * q**(n-i)


In [45]:
std = 0
for i in range(n+1):
    pi = np.math.comb(n, i) * p**i * q**(n-i)
    std += ((i/3 - mu)**2) * pi
    
print("Standard deviation is: ", np.sqrt(std))

Standard deviation is:  0.24195029428289866


/var/folders/h6/9mpjzmxd77l7493g4210phz00000gn/T/ipykernel_1008/2788258256.py:3: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  pi = np.math.comb(n, i) * p**i * q**(n-i)


### `2c`
Same question as (b) but without replacement after each deal of the river card. Is the standard deviation larger or smaller than the standard deviation in part (b), and does that make sense?

- One approach is to calculate the probability distribution of Patrik’s winnings. 
- The probability that Patrik wins exactly 0/3 of the full pot is
$$\frac{\binom{34}{0}\binom{10}{3}}{\binom{44}{3}}$$ 
- The probability that Patrik wins exactly 1/3 of the full pot is
$$\frac{\binom{34}{1}\binom{10}{2}}{\binom{44}{3}}$$ 
- The probability that Patrik wins exactly 2/3 of the full pot is
$$\frac{\binom{34}{2}\binom{10}{1}}{\binom{44}{3}}$$ 
- The probability that Patrik wins exactly 3/3 of the full pot is
$$\frac{\binom{34}{3}\binom{10}{0}}{\binom{44}{3}}$$ 
- Without replacement, the new expetation of Patrik's winning is,
$$\begin{aligned}E[X]&=\sum X_i \cdot p(X_i) \\
&= 0.773 \\
\end{aligned}$$
- The new variance of Patrik's winning is,
$$\begin{aligned}
Var(X)&=E[X^2]- E[X]^2 \\
&=\sum \left[X_i^2 \cdot p(X_i)\right] - \left[\sum X_i \cdot p(X_i)\right]^2 \\
\end{aligned}$$
- The new standard deviation of Patrik's winning is,
$$\sigma(X) = \sqrt{Var(X)}=0.236$$
The standard deviation in part c is smaller than in part b, which makes sense because the probability varies less.

In [50]:
win = 34
lose = 10
n = 3

mu = 0
for i in range(n+1):
    pi = np.math.comb(win, i) * np.math.comb(lose, n-i) / np.math.comb(win+lose, n)
    mu += (i/3) * pi
    
print("Mean is: ", mu)

Mean is:  0.7727272727272727


/var/folders/h6/9mpjzmxd77l7493g4210phz00000gn/T/ipykernel_1008/2171101861.py:7: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  pi = np.math.comb(win, i) * np.math.comb(lose, n-i) / np.math.comb(win+lose, n)


In [52]:
std = 0
for i in range(n+1):
    pi = np.math.comb(win, i) * np.math.comb(lose, n-i) / np.math.comb(win+lose, n)
    std += ((i/3 - mu)**2) * pi
    
print("Standard deviation is: ", np.sqrt(std))

Standard deviation is:  0.23625654862570683


/var/folders/h6/9mpjzmxd77l7493g4210phz00000gn/T/ipykernel_1008/369237363.py:3: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  pi = np.math.comb(win, i) * np.math.comb(lose, n-i) / np.math.comb(win+lose, n)
